In [3]:
import ijson
from tqdm.auto import tqdm

path = "../../data/news-all-final.json"
num_comments = 0
num_articles = 537_000
data = []

with open(path) as f:
    articles = ijson.items(f, 'item')
    for article in tqdm(articles, total=num_articles):
        num_comments += len(article["comments"])

        context = article["title"] if "title" in article else article["text"]

        for comment in article["comments"]:
            data.append({
                "user": article["user"],
                "text": comment["text"],
                "context": context,
                "id": comment["tweet_id"],
            })

    print(num_comments, " comments processed")


8569648  comments processed


In [66]:
import glob 
import pandas as pd

df_comments = pd.DataFrame(data)
labels = pd.concat([pd.read_csv(path) for path in glob.glob("../../data/predictions/*.csv")])
del labels["Unnamed: 0"]

df_comments.set_index("id", inplace=True)
labels.set_index("id", inplace=True)


In [67]:
from hatedetection import extended_hate_categories

df = labels.merge(df_comments, left_index=True, right_index=True)
df = df.reindex(columns=[ 'user', 'text', 'context'] + extended_hate_categories)

In [68]:
df["HATEFUL"] = (df[extended_hate_categories[1:]].sum(axis=1) > 0)

df.shape, df[extended_hate_categories].sum()

((800754, 13),
 CALLS          3631
 WOMEN          7248
 LGBTI          1994
 RACISM         7197
 CLASS          2838
 POLITICS      10882
 DISABLED       4918
 APPEARANCE    13813
 CRIMINAL       4392
 dtype: int64)

In [69]:
df["user"].value_counts()

clarincom          157273
elmundoes          145288
infobae            140244
LANACION           117771
abc_es              74484
latercera           71796
elpaisuy            27789
LaVanguardia        26140
cronica             18282
perfilcom           14339
laderechadiario      7312
pagina12               25
izquierdadiario        11
Name: user, dtype: int64

In [71]:
df[df["user"] == "laderechadiario"][extended_hate_categories].sum()

CALLS          55
WOMEN          41
LGBTI          19
RACISM        113
CLASS          17
POLITICS      291
DISABLED       30
APPEARANCE     80
CRIMINAL       16
dtype: int64

In [74]:
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 500
pd.options.display.min_rows = 300
df[ df["RACISM"] & (df["user"] == "laderechadiario")][["text", "context"]]

,text,context
id,,
1242881585369800706,@laderechadiario Jajajajajajaja virus chino que hijos de pu,Putin saca ventaja del caos en Europa por el coronavirus
1242921337653473280,"@laderechadiario @noticias_taxi Que hdps los chinos de mierda, nos enferman y dsp nos quieren vender la cura",¡Gracias @noticias_taxi por utilizar una nota nuestra! Un orgullo que el contenido de La Derecha Diario llegue a todo Latinoamérica. https://t.co/HZgqdnSATM
1245060221271191553,"@laderechadiario Quien en su sano juicio puede seguir haciendo negocios con estos genocidas... ya han ido demasiado lejos y para colmo reabren los mercados donde supuestamente salio el virus, se burlan de nosotros!.",En China habrían muerto entre 10 y 20 veces más que los números oficiales por el coronavirus
1245084028618919939,@laderechadiario Estos chinos comunistas deberían ser enjuiciados por genocidio,En China habrían muerto entre 10 y 20 veces más que los números oficiales por el coronavirus
1245098577170436099,"@laderechadiario China ocultó la enfermedad, ocultó información y evidencia, censuró a todo habitante que quiso alertar sobre el coronavirus. China es responsable de todas las muertes que están ocurriendo en el mundo. Mao mató a más de 80 millone...",En China habrían muerto entre 10 y 20 veces más que los números oficiales por el coronavirus
1245159273241497601,"@laderechadiario Los chinos son tan miserables y anormales que ni la vida de su propia gente valoran ni respetan, parece que se ven así mismos como bestias unos a otros. Que decadencia moral caray!!",En China habrían muerto entre 10 y 20 veces más que los números oficiales por el coronavirus
1245131206976507904,@laderechadiario Solo les queda tomar vino en copa de plomo para ser tanto o más pelotudo que los romanos del imperio.,Entre el delirio y el oportunismo: el encuentro del Presidente con Residente
1245863728156782593,"@laderechadiario Pobre el que se creyo que en un pais mas rico, mas limpio y menos poblado podia haber mas muertos que en wuhan. Imaginense lo que es india ahora!",Fuimos los primeros en publicar sobre las mentiras del régimen chino por las muertes de coronavirus y nos trataron de fake news.\n\n¿Dirán lo mismo ahora que los principales medios replicaron nuestra noticia? https://t.co/Qw8pIkmmN2
1245896957559799810,"@laderechadiario Comunista hijo de remil puta, inepto y basura. Me quiero ir de este pais del orto",El gobierno quiso expropiar las clínicas privadas en medio de la pandemia pero no se animó


In [ ]:
df.to_csv("comentarios_analizados.csv")